In [ ]:
from dotenv import load_dotenv
load_dotenv()
import openai, os
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = os.environ["OPENAI_API_BASE"]
openai.api_key = os.environ["OPENAI_API_KEY"]

print("OpenAI Endpoint:", openai.api_base)
print("MLFlow Tracking URI: ", os.getenv("MLFLOW_TRACKING_URI"))


In [ ]:
from src.langchain.patch import patch_langchain
import mlflow

patch_langchain()
mlflow.end_run()

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI, AzureOpenAI
from langchain import OpenAI, LLMMathChain, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

chat = AzureChatOpenAI(
    deployment_name="gpt-4",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)
llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]
tools = tools + load_tools(["requests_get"])
mrkl = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

mrkl.run("how much money would I have if I had invested 1000 Dollars in MSFT 2 years ago? Today's date is 2023-04-17.")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI

chat = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    openai_api_version="2023-03-15-preview",
)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=chat, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("organic snacks"))